In [1]:
#Importing nessecary libraries and MPS_MPO.py
import numpy as np
import math
from numpy import linalg as LA
from scipy import linalg as SLA
from MPS_MPO_fix import *

## First Let's try our MPS function on random state

In [2]:
#Let's try for random states
sshape = (2,3,4,5,6)
c2 = np.random.rand(*sshape) + np.random.rand(*sshape) * 1j
c2 = c2/np.sqrt(np.sum(np.conj(c2) * c2))

In [3]:
#Apply MPS function to the state c2 (Print the size of Gamma Gamma_{alpha_{l-1},i_l, alpha_l}
G,S = MPS(c2)

[(2, 2), (2, 3, 6), (6, 4, 24), (24, 5, 6), (6, 6)]


The $G$ is a list consist of $\Gamma^{[l] i_l}_{\alpha_{l-1} \alpha_l}$ and the index order of element in $G$ is $[\alpha_{l-1}, i_l, \alpha_{l}]$. And similarly, $S$ is a list consist of $\lambda^{[l]}_{\alpha_l}$.

In [4]:
G[2].shape

(6, 4, 24)

Now, we want to recreate the state from the MPS we calculated. So we directly calculated $recreate = \Gamma^{[0] i_0}_{\alpha_0} \lambda^{[0]}_{\alpha_0}\Gamma^{[1] i_1}_{\alpha_0,\alpha_1} \cdots \lambda^{[N-1]}_{\alpha_{N-1}} \Gamma^{[N] i_N}_{\alpha_{N-1}}$.

Then we calculate the Frobenius norm (sum of square of all elements of the tensor) of original state and norm of difference between original state and recreate state.

In [5]:
#Checking if the MPS can regenerate the original state
recreate = G[0]
for i in range(1, len(c2.shape)):
    recreate = np.tensordot(recreate, np.diag(S[i-1]), (-1,0))
    recreate = np.tensordot(recreate, G[i], (-1,0))
print('The original state(c2) norm',np.sqrt(np.sum(np.conj(c2) * c2)))
print('c2 - MPS norm',np.sqrt(np.sum(np.conj(c2-recreate) * (c2-recreate))))

The original state(c2) norm (1.0000000000000002+0j)
c2 - MPS norm (9.225986565864108e-15+0j)


Since we have confirm we can recreate the state from MPS, now we want to check if we can get Schimidt's decomposition for arbitrary $l$ that $|\Phi\rangle = \lambda^{[l]}_{\alpha_l} |\Phi^{[1,\cdots,l]}_{\alpha_l}\rangle |\Phi^{[l+1, \cdots, N]}_{\alpha_l}\rangle$ where we have:
$$|\Phi^{[1,\cdots,l]}_{\alpha_l}\rangle = \Gamma^{[0] i_0}_{\alpha_0} \lambda^{[0]}_{\alpha_0} \cdots \lambda^{[l-1]}_{\alpha_{l-1}} \Gamma^{[l] i_l}_{\alpha_{l-1}, \alpha_{l}} |i_0,i_1,\cdots, i_l\rangle$$ and similarly
$$|\Phi^{[l+1,\cdots,N]}_{\alpha_l}\rangle = \Gamma^{[l+1] i_{l+1}}_{\alpha_l, \alpha_{l+1}} \lambda^{[l+1]}_{\alpha_{l+1}} \cdots \lambda^{[N-1]}_{\alpha_{N-1}} \Gamma^{[N] i_N}_{\alpha_{N-1}} |i_0,i_1,\cdots, i_l\rangle$$

From previous step, we have already checked that $|\Phi\rangle = \lambda^{[l]}_{\alpha_l} |\Phi^{[1,\cdots,l]}_{\alpha_l}\rangle |\Phi^{[l+1, \cdots, N]}_{\alpha_l}\rangle$, now we only need to check if $|\Phi^{[1,\cdots,l]}_{\alpha_l}\rangle$ and $|\Phi^{[l+1, \cdots, N]}_{\alpha_l}\rangle$ are unit vectors. We also need to check if $\sum_{\alpha_l} (\lambda^{[l]}_{\alpha_l})^2 = 1$.

The reason we need to this Schimidt's decomposition property to work is that we can later calculate the evolope function in $O(\chi^2 d)$ time complexity and $O(\chi d)$ memory complexity.

In [6]:
#Checking if it extract Schimdt's decomposition for lambda_i |phi
l = 1
N = len(c2.shape)
left = G[0]
right = G[l+1]
for i in range(1,l+1):
    left = np.tensordot(left, np.diag(S[i-1]),(-1,0))
    left = np.tensordot(left, G[i],(-1,0))
for j in range(l+1,N-1):
    right = np.tensordot(right, np.diag(S[j]),(-1,0))
    right = np.tensordot(right, G[j+1], (-1,0))
#Checking the normality of left and right
print("The norms for the left part and its shape:")
for j in range(len(S[l])):
    temp = left[..., j]
    print(np.sum(np.conj(temp)*temp), temp.shape)
    #print(left)
print("Now, let's print the norm for the right part:")
for j in range(len(S[l])):
    temp = right[j]
    print(np.sum(np.conj(temp)*temp), temp.shape)

print("And this is the Lambdas:")
print(S[l], np.sum(np.power(S[l],2)))

The norms for the left part and its shape:
(1+0j) (2, 3)
(1+0j) (2, 3)
(0.9999999999999999+0j) (2, 3)
(1+0j) (2, 3)
(1.0000000000000002+0j) (2, 3)
(1+0j) (2, 3)
Now, let's print the norm for the right part:
(1.0000000000000009+0j) (4, 5, 6)
(0.9999999999999993+0j) (4, 5, 6)
(0.9999999999999987+0j) (4, 5, 6)
(0.9999999999999973+0j) (4, 5, 6)
(0.9999999999999996+0j) (4, 5, 6)
(0.999999999999999+0j) (4, 5, 6)
And this is the Lambdas:
[0.8907624  0.22831986 0.20009983 0.18629031 0.21183353 0.18653399] 0.9999999999999999


Notice, for fixed $\alpha_{l-1}$ and $\alpha_{l}$, we can not gareented $\Gamma^{[l] i_{l}}_{\alpha_{l-1}, \alpha_{l}}$ is unit vector (The following is an example: $\Gamma^{[1] i_{1}}_{\alpha_{0} = 1, \alpha_{l} = 0}$ is not unit vector).

In [7]:
vec = G[2][1,:,0]
np.sum(np.conj(vec)*vec)

(0.026093611333529255+0j)

## Now let's try the same thing on slightly entangled state with fixed spin dimension 2

In [8]:
#Generate the slightly local entangled State
c = np.zeros((2,2,2,2,2,2), dtype = np.complex_)
c[(0,0,0,0,0,0)] += 1.
c[(1,1,0,0,0,0)] += 1/3.
c[(0,0,1,1,0,0)] += 1/4.
#c[(0,1,1,0,0,0)] += 1/5+0.2j
c[(0,0,0,0,1,1)] += 1/6.
#c[(0,0,0,1,1,0)] += 1/7.
#Normailize the State
c = c/np.sqrt(np.sum(np.conj(c) * c))

In [9]:
#Apply MPS function to the state c (Print the size of Gamma Gamma_{alpha_{l-1},i_l, alpha_l}
G,S = MPS(c)

[(2, 2), (2, 2, 2), (2, 2, 3), (3, 2, 2), (2, 2, 2), (2, 2)]


In [10]:
#Checking if the MPS can regenerate the original state
recreate1 = G[0]
for i in range(1, len(c.shape)):
    recreate1 = np.tensordot(recreate1, np.diag(S[i-1]), (-1,0))
    recreate1 = np.tensordot(recreate1, G[i], (-1,0))
print('The original state(c) norm',np.sqrt(np.sum(np.conj(c) * c)))
print('c - MPS norm',np.sqrt(np.sum(np.conj(c-recreate1) * (c-recreate1))))

The original state(c) norm (1+0j)
c - MPS norm (7.470130247756179e-16+0j)


In [11]:
#Checking if we can extract Schimdt's decomposition from it for lambda_i |phi^[1...l]>|phi^[l+1...N]>
l = 2
N = len(c.shape)
left = G[0]
right = G[l+1]
for i in range(1,l+1):
    left = np.tensordot(left, np.diag(S[i-1]),(-1,0))
    left = np.tensordot(left, G[i],(-1,0))
for j in range(l+1,N-1):
    right = np.tensordot(right, np.diag(S[j]),(-1,0))
    right = np.tensordot(right, G[j+1], (-1,0))
#Checking the normality of left and right
print("The norms for the left part and its shape:")
for j in range(len(S[l])):
    temp = left[..., j]
    print(np.sum(np.conj(temp)*temp), temp.shape)
print("Now, let's print the norm for the right part:")
for j in range(len(S[l])):
    temp = right[j]
    print(np.sum(np.conj(temp)*temp), temp.shape)
print("And this is the Lambdas:")
print(S[l])

The norms for the left part and its shape:
(1.0000000000000002+0j) (2, 2, 2)
(1+0j) (2, 2, 2)
(1+0j) (2, 2, 2)
Now, let's print the norm for the right part:
(1.0000000000000004+0j) (2, 2, 2)
(0.9999999999999996+0j) (2, 2, 2)
(1+0j) (2, 2, 2)
And this is the Lambdas:
[0.96169407 0.22808578 0.15205718]


## Now let's try the two mode unitary operator operate on MPS

In [12]:
#Now let's try the two mode unitrary operator
#First we generate a (real) Hermitian operator for two mode
V = np.zeros((2,2,2,2), dtype = np.complex_)
#V[(1,0,0,1)] += 1/2
#V[(0,1,1,0)] += 1/2
V[(1,0,1,0)] += 1/3
V[(0,1,0,1)] += 1/4
V[(0,1,0,1)] += 1/5
V[(1,1,1,1)] += 1/6
V[(0,0,0,0)] += 1/7
#Time evolution operator U for deltat = 0.1
Vr= V.reshape(4,4)
#print("V - V hermitian is: ", Vr - np.transpose(np.conj(Vr)))
deltat = 0.1
U = SLA.expm(- 1.j * deltat *Vr).reshape(2,2,2,2)
#Check whether U is unitary
Ur = U.reshape(4,4)
print("U", Ur)
print("U times U hermitian is: ", Ur*np.transpose(np.conj(Ur)))
G_0 = np.copy(G); S_0 = np.copy(S)
G_new, S_new = two_mode(G_0,S_0,U,3)

U [[0.99989796-0.01428523j 0.        +0.j         0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.99898767-0.04498481j 0.        +0.j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.9994445 -0.03332716j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.j         0.        +0.j
  0.99986111-0.0166659j ]]
U times U hermitian is:  [[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]]


C:\Users\yukai\anaconda3\lib\site-packages\numpy\lib\function_base.py:793: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, order=order, subok=subok, copy=True)


In [13]:
#Checking if we can extract Schimdt's decomposition from it for lambda_i |phi^[1...l]>|phi^[l+1...N]>
l = 3
N = len(c.shape)
left = G_new[0]
right = G_new[l+1]
for i in range(1,l+1):
    left = np.tensordot(left, np.diag(S_new[i-1]),(-1,0))
    left = np.tensordot(left, G_new[i],(-1,0))
for j in range(l+1,N-1):
    right = np.tensordot(right, np.diag(S_new[j]),(-1,0))
    right = np.tensordot(right, G_new[j+1], (-1,0))
#Checking the normality of left and right
print("The norms for the left part and its shape:")
for j in range(len(S_new[l])):
    temp = left[..., j]
    print(np.sum(np.conj(temp)*temp), temp.shape)
    temp2 = np.tensordot(np.diag(S_new[l-1]),G_new[l],(0,0))[...,j]
    #temp2 = np.tensordot(G_new[l-1],temp2,(-1,0))
    #print(temp2.shape,len(S_new[l-2]))
    #temp2 = np.tensordot(np.diag(S_new[l-2]),temp2,(-1,0))[...,j]
    print(np.sum(np.conj(temp2)*temp2), temp2.shape)
print("Now, let's print the norm for the right part:")
for j in range(len(S_new[l])):
    temp = right[j]
    print(np.sum(np.conj(temp)*temp), temp.shape)
print("And this is the Lambdas:")
print(S_new[l], np.sum(np.power(S_new[l],2)))

The norms for the left part and its shape:
(0.9999999999999999+0j) (2, 2, 2, 2)
(0.9999999999999998+0j) (3, 2)
(1+0j) (2, 2, 2, 2)
(1+0j) (3, 2)
Now, let's print the norm for the right part:
(1+0j) (2, 2)
(0.9999999999999998+0j) (2, 2)
And this is the Lambdas:
[0.9883717  0.15205718] 1.0000000000000018


Additionally, we need to check the orthogonality of Schmidt's Decomposition

In [14]:
cl = G_new[0]
for i in range(1, 2):
    cl = np.tensordot(cl, np.diag(S_new[i-1]), (-1,0))
    cl = np.tensordot(cl, G_new[i], (-1,0))
print([np.conj(cl[...,0])*cl[...,1]])

[array([[-0.90751445-0.j,  0.        +0.j],
       [ 0.        +0.j,  0.09248555+0.j]])]


## Now we want to calculate the possibilty for one spin to be some state (|0> or |1>).

We have a fast method if the previous Schimidt's decomposition is correct:
$$P(|i_l\rangle = |k\rangle) = (norm( \lambda^{[l-1]}_{\alpha_{l-1}} \Gamma^{[l] i_l = k}_{\alpha_{l-1}, \alpha_{l}} \lambda^{[l]}_{\alpha_{l}}))^2 = \sum_{\alpha_{l-1}, \alpha_{l}} (\lambda^{[l-1]}_{\alpha_{l-1}} \Gamma^{[l] i_l = k}_{\alpha_{l-1}, \alpha_{l}} \lambda^{[l]}_{\alpha_{l}})^2$$

In [15]:
a = c[0,...]
print("The actrual possiblity of spin_0 be 0 is:", np.sum(np.conj(a)*a))
aa = np.tensordot(G[0][0], np.diag(S[0]),(-1,0))
print("The possiblity calculatd by the fast method:", np.sum(np.conj(aa)*aa))

a = c[:,1,...]
print("The actrual possiblity of spin_1 be 1 is:", np.sum(np.conj(a)*a))
aa = np.tensordot(np.diag(S[0]),np.tensordot(G[1][:,1,:], np.diag(S[1]),(-1,0)),(-1,0))
print("The possiblity calculatd by the fast method:", np.sum(np.conj(aa)*aa))

a = c[:,:,:,0,...]
print("The actrual possiblity of spin_3 be 0 is:", np.sum(np.conj(a)*a))
aa = np.tensordot(np.diag(S[2]),np.tensordot(G[3][:,0,:], np.diag(S[3]),(-1,0)),(-1,0))
print("The possiblity calculatd by the fast method:",np.sum(np.conj(aa)*aa))



The actrual possiblity of spin_0 be 0 is: (0.9075144508670521+0j)
The possiblity calculatd by the fast method: (0.907514450867052+0j)
The actrual possiblity of spin_1 be 1 is: (0.09248554913294797+0j)
The possiblity calculatd by the fast method: (0.092485549132948+0j)
The actrual possiblity of spin_3 be 0 is: (0.9479768786127167+0j)
The possiblity calculatd by the fast method: (0.9479768786127183+0j)


In [16]:
a = c[0,...]; b = c[1,...]
print("The actrual c_i*c_j is:", np.sum(np.conj(a)*b))
aa = np.tensordot(G[0][0], np.diag(S[0]),(-1,0))
bb = np.tensordot(G[0][1], np.diag(S[0]),(-1,0))
print("The possiblity calculatd by the fast method:", np.sum(np.conj(aa)*bb))

a = c[:,1,...]; b = c[:,0,...]
print("The actrual possiblity of spin_1 be 1 is:", np.sum(np.conj(a)*b))
aa = np.tensordot(np.diag(S[0]),np.tensordot(G[1][:,1,:], np.diag(S[1]),(-1,0)),(-1,0))
bb = np.tensordot(np.diag(S[0]),np.tensordot(G[1][:,0,:], np.diag(S[1]),(-1,0)),(-1,0))
print("The possiblity calculatd by the fast method:", np.sum(np.conj(aa)*bb))


The actrual c_i*c_j is: 0j
The possiblity calculatd by the fast method: 0j
The actrual possiblity of spin_1 be 1 is: 0j
The possiblity calculatd by the fast method: 0j


In [17]:
crec = G_new[0]
for i in range(1, len(c.shape)):
    crec = np.tensordot(crec, np.diag(S_new[i-1]), (-1,0))
    crec = np.tensordot(crec, G_new[i], (-1,0))

a = crec[0,...]
print(np.sum(np.conj(a)*a))
aa = np.tensordot(G_new[0][0], np.diag(S_new[0]),(-1,0))
print(np.sum(np.conj(aa)*aa))

a = crec[:,1,...]
print(np.sum(np.conj(a)*a))
aa = np.tensordot(np.diag(S[0]),np.tensordot(G_new[1][:,1,:], np.diag(S_new[1]),(-1,0)),(-1,0))
print(np.sum(np.conj(aa)*aa))

a = crec[:,:,0,...]
print(np.sum(np.conj(a)*a))
aa = np.tensordot(np.diag(S_new[1]),np.tensordot(G_new[2][:,0,:], np.diag(S_new[2]),(-1,0)),(-1,0))
print(np.sum(np.conj(aa)*aa))

a = crec[:,:,1,...]
print(np.sum(np.conj(a)*a))
aa = np.tensordot(np.diag(S_new[1]),np.tensordot(G_new[2][:,1,:], np.diag(S_new[2]),(-1,0)),(-1,0))
print(np.sum(np.conj(aa)*aa))


a = crec[:,:,:,0,...]
print(np.sum(np.conj(a)*a))
aa = np.tensordot(np.diag(S_new[2]),np.tensordot(G_new[3][:,0,:], np.diag(S_new[3]),(-1,0)),(-1,0))
print(np.sum(np.conj(aa)*aa))



a = crec[:,:,:,:,0,...]
print(np.sum(np.conj(a)*a))
aa = np.tensordot(np.diag(S_new[3]),np.tensordot(G_new[4][:,0,:], np.diag(S_new[4]),(-1,0)),(-1,0))
print(np.sum(np.conj(aa)*aa))

(0.9075144508670538+0j)
(0.907514450867052+0j)
(0.09248554913294815+0j)
(0.092485549132948+0j)
(0.9479768786127187+0j)
(0.9713375796178355+0j)
(0.05202312138728324+0j)
(0.028662420382165616+0j)
(0.9479768786127187+0j)
(0.9479768786127183+0j)
(0.9768786127167649+0j)
(0.9768786127167648+0j)


In [18]:
a = np.random.rand(2,3,4)
b = np.random.rand(4,5,6)
print('Norm of a is', np.sum(np.power(a,2)))
print('Norm of b is', np.sum(np.power(b,2)))
c = np.tensordot(a,b,(-1,0))
print('Norm of c is', np.sum(np.power(c,2)))

Norm of a is 7.383267255569606
Norm of b is 40.52764630063643
Norm of c is 204.3474611905113
